In [44]:
import pandas as pd
import matplotlib.pyplot as plt
#import seaborn as sns
path_gff_tes = 'data/results/all.noun.no00.gff'
path_gff_genes = 'data/results/Stuberosum_genes_sin_chr0.gff3'
path_genome = 'data/Stuberosum_genome.gff3'
te_classes = ['TRIM', 'LARD','LTR','SINE', 'LINE','MITE', 'TIR','helitron']

In [45]:
df_genome = pd.read_csv(path_genome, sep='\t', header=None)
df_genome.columns = ['seqname' , 'source' , 'feature' , 'start' , 'end' , 'score' , 'strand' , 'frame' , 'attribute']

df_tes = pd.read_csv(path_gff_tes, sep='\t', header=None)
df_tes.columns = ['seqname' , 'source' , 'feature' , 'start' , 'end' , 'score' , 'strand' , 'frame' , 'attribute']

df_genes = pd.read_csv(path_gff_genes, sep='\t', header=None)
df_genes.columns = ['seqname' , 'source' , 'feature' , 'start' , 'end' , 'score' , 'strand' , 'frame' , 'attribute']


In [46]:
print(len(df_tes.index))

227947


In [47]:
df_genes['new_start'] = df_genes[['start','end']].min(axis=1)
df_genes['new_send'] = df_genes[['start','end']].max(axis=1)
df_genes['start'] = df_genes['new_start']
df_genes['end'] = df_genes['new_send']
df_genes = df_genes.drop('new_start',axis=1).drop('new_send',axis=1)
df_genes.sort_values(['seqname', 'start'], inplace=True)

In [48]:
print(len(df_genes.index))

37482


In [49]:
#load genome structure
df_genes_genome = {}
for k,chromosome in df_genome.iterrows():
    df_genes_genome[chromosome.seqname] = df_genes[(df_genes.seqname == chromosome.seqname)].reset_index()

In [50]:
distances_kb = {}
distances_kb_filtered = {}
for te_class in te_classes:
    touch_gene = 0
    #calculate distance to closest gene
    df_tes_class = df_tes[df_tes.attribute.str.contains(te_class + "_id")]
    for k,v in df_tes_class.iterrows():
        df_curr = df_genes_genome[v.seqname]
        df_curr = df_curr[(df_curr.end >= v.start)  & (df_curr.start <= v.end )]
        if len(df_curr.index) > 0:
            touch_gene += 1
    total_len = len(df_tes_class.index)
    print(te_class, total_len, touch_gene)

TRIM 11630 748
LARD 5 3
LTR 172509 15550
SINE 2413 539
LINE 164 48
MITE 36854 2604
TIR 3264 328
helitron 1108 262


In [ ]:
for te_class in te_classes:
    # seaborn histogram
    plt.figure()
 
    sns.distplot(distances_kb_filtered[te_class], hist=True, kde=False, 
                 bins=80, color = 'blue',
                 hist_kws={'edgecolor':'black'},rug=False)
    # Add labels
    plt.title(te_class + ' distance to closest gene in the genome')
    plt.xlabel('Distance to closest gene (kb)')
    plt.ylabel('Number of ' + te_class)
    plt.savefig('data/results/plots/histogram' + te_class + '.png', dpi=800,bbox_inches='tight')



['TRIM', 'LARD', 'LTR', 'SINE', 'LINE', 'MITE', 'TIR', 'helitron']